Задание №1. Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:

- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
import pandas as pd

In [4]:
ratings = pd.read_csv('ratings11.csv')
ratings.head(30)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [5]:
def stage(score):
    if score <= 2:
        return 'bottom'
    elif score > 2 and score <=4:
        return 'middle'
    else:
        return 'top' 
ratings['level'] = ratings['rating'].apply(stage)
ratings.head(30)

,userId,movieId,rating,timestamp,level
0,1,31,2.5,1260759144,middle
1,1,1029,3.0,1260759179,middle
2,1,1061,3.0,1260759182,middle
3,1,1129,2.0,1260759185,bottom
4,1,1172,4.0,1260759205,middle
5,1,1263,2.0,1260759151,bottom
6,1,1287,2.0,1260759187,bottom
7,1,1293,2.0,1260759148,bottom
8,1,1339,3.5,1260759125,middle
9,1,1343,2.0,1260759131,bottom


Задание №2. Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [6]:
keywords = pd.read_csv('keywords11.csv')
keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [7]:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [8]:
def region(str):
    list_str = str.split(' ')
    set_str = set(list_str)
    for key,value in geo_data.items():
        geo = set(value)
        if set_str & geo == set():
            return 'none'
        else:
            return key

keywords['region'] = keywords['keyword'].apply(region)
keywords.head()

,keyword,shows,region
0,вк,64292779,none
1,одноклассники,63810309,none
2,порно,41747114,none
3,ютуб,39995567,none
4,вконтакте,21014195,none


Задание 3. Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
	- для каждой строки пройдите по всем годам списка years
	- если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
	- если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [9]:
ratings = pd.read_csv('ratings11.csv')
ratings.head(10)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [10]:
movies = pd.read_csv('movies11.csv')
movies.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [11]:
years= [i for i in range(1950,2011)]
years

[1950,
 1951,
 1952,
 1953,
 1954,
 1955,
 1956,
 1957,
 1958,
 1959,
 1960,
 1961,
 1962,
 1963,
 1964,
 1965,
 1966,
 1967,
 1968,
 1969,
 1970,
 1971,
 1972,
 1973,
 1974,
 1975,
 1976,
 1977,
 1978,
 1979,
 1980,
 1981,
 1982,
 1983,
 1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010]

In [12]:
def release_year(str_title):
    for x in years:
        if str_title.find(str(x)) > 0: 
            return x
    return 1990
        
movies['year'] = movies['title'].apply(release_year)
movies.head(10)

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995
5,6,Heat (1995),Action|Crime|Thriller,1995
6,7,Sabrina (1995),Comedy|Romance,1995
7,8,Tom and Huck (1995),Adventure|Children,1995
8,9,Sudden Death (1995),Action,1995
9,10,GoldenEye (1995),Action|Adventure|Thriller,1995


In [13]:
movies['year'].unique()

array([1995, 1994, 1996, 1976, 1992, 1967, 1993, 1964, 1977, 1965, 1982,
       1990, 1991, 1989, 1969, 1981, 1973, 1970, 1955, 1959, 1968, 1988,
       1997, 1972, 1952, 1951, 1957, 1961, 1958, 1954, 1960, 1963, 1953,
       1950, 1956, 1975, 1974, 1971, 1979, 1987, 1986, 1980, 1978, 1985,
       1966, 1962, 1983, 1984, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010], dtype=int64)

In [18]:
join = ratings.merge(movies)
join.head(10)

,userId,movieId,rating,timestamp,title,genres,year
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,1995
1,7,31,3.0,851868750,Dangerous Minds (1995),Drama,1995
2,31,31,4.0,1273541953,Dangerous Minds (1995),Drama,1995
3,32,31,4.0,834828440,Dangerous Minds (1995),Drama,1995
4,36,31,3.0,847057202,Dangerous Minds (1995),Drama,1995
5,39,31,3.0,832525157,Dangerous Minds (1995),Drama,1995
6,73,31,3.5,1255591860,Dangerous Minds (1995),Drama,1995
7,88,31,3.0,1239755559,Dangerous Minds (1995),Drama,1995
8,96,31,2.5,1223256331,Dangerous Minds (1995),Drama,1995
9,110,31,4.0,840100695,Dangerous Minds (1995),Drama,1995


In [19]:
join.groupby('year').mean().sort_values(by='rating', ascending=False)

,userId,movieId,rating,timestamp
year,,,,
1957,360.149153,2122.172881,4.067797,1.082142e+09
1952,336.184615,2073.792308,4.061538,1.080898e+09
1954,354.685015,2136.259939,4.044343,1.066002e+09
1972,358.253555,2725.725118,4.029621,1.120346e+09
1974,353.555126,2770.458414,4.001934,1.105290e+09
1951,345.612766,1674.136170,3.991489,1.047464e+09
1950,350.191964,2053.678571,3.977679,1.085261e+09
1962,348.603933,3131.550562,3.957865,1.088881e+09
1977,350.202435,2822.242009,3.911720,1.113257e+09
